<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use Spark and Python to Predict Equipment Purchase</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Watson Studio Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to the Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and Apache Spark 2.1.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment and so on.

**Note**: In this notebook we use GOSales data available on [Watson Studio Community](https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600).

## Learning goals

You will learn how to:

-  Load a CSV file into an Apache Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache Spark machine learning pipeline.
-  Train and evaluate a model.
-  Store a pipeline and model in Watson Machine Learning (WML) repository.
-  Deploy a model for online scoring using the Watson Machine Learning (WML) API.
-  Score sample scoring data using the WML API.
-  Explore and visualize the prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Set up](#setup)
2.	[Load and explore data](#load)
3.	[Create a spark ml model](#model)
4.	[Store the model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a lite plan is offered and information about how to create the instance is <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>)
-  Make sure that you are using a Spark 2.1 kernel
-  Download **GoSales Transactions** from the Watson Studio Community (code provided below)

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then use spark *read* method to read data. 

In [1]:
# Install wget if you don't already have it.
!pip install wget

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181005044014-0001
  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/spark/shared/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:


import wget

link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = wget.download(link_to_data)

print(filename)



GoSales_Tx_NaiveBayes.csv


In [3]:



spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)



The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache Spark DataFrame using code below.

Explore the loaded data by using the following Apache Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [4]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one you would like to predict (label).

In [5]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [6]:
print("Number of records: " + str(df_data.count()))

Number of records: 60252


As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)

### 3.1 Prepare data<a id="prep"></a>

In this subsection you will split your data into: 
- train data set
- test data set
- predict data set

In [7]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


As you can see your data has been successfully split into three data sets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache Spark machine learning packages that will be needed in the subsequent steps.

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [9]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [10]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [11]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, convert the indexed labels back to original labels.

In [12]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Now build the pipeline. A pipeline consists of transformers and an estimator.

In [13]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

### 3.3 Train the model<a id="train"></a>

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [14]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [15]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. Use **test data** to evaluate the model.

In [16]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.582965
Test Error = 0.417035


You can tune your model now to achieve better accuracy. For simplicity, the tuning example is omitted in this example.

<a id="persistence"></a>
## 4. Store the model in the WML repository

In this section you will learn how to use Python client libraries to store your pipeline and model in WML repository.

- [4.1 Import the libraries](#lib)
- [4.2 Save pipeline and model](#save)
- [4.3 Load the model](#load)

### 4.1 Import the libraries<a id="lib"></a>

First, you must install and import the `watson-machine-learning-client` libraries.

**Note**: Python 3.5 and Apache Spark 2.1 is required.

In [17]:
#!rm -rf $PIP_BUILD/watson-machine-learning-client

In [21]:
!pip install watson-machine-learning-client --upgrade

    100% |################################| 942kB 785kB/s eta 0:00:01
    100% |################################| 8.7MB 111kB/s eta 0:00:01
    100% |################################| 51kB 1.3MB/s eta 0:00:01
    100% |################################| 153kB 1.9MB/s eta 0:00:01
    100% |################################| 51kB 1.6MB/s eta 0:00:01
    100% |################################| 92kB 2.0MB/s eta 0:00:01
    100% |################################| 143kB 2.1MB/s eta 0:00:01
    100% |################################| 51kB 1.2MB/s eta 0:00:01
    100% |################################| 215kB 1.8MB/s eta 0:00:01
    100% |################################| 512kB 1.2MB/s eta 0:00:01
    100% |################################| 13.8MB 71kB/s  eta 0:00:01
    100% |################################| 788kB 912kB/s eta 0:00:01
    100% |################################| 194kB 2.0MB/s eta 0:00:01
    100% |################################| 61kB 1.6MB/s eta 0:00:01
    100% |##############

In [22]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on IBM Cloud. 

If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [23]:
wml_credentials={
  "apikey": "JrmdFswCQfOOgeknmUtXT_rfzqMLHjM1v1D2WEf2unfb",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:eu-de:a/729dbd0074f3e622e009b8be41b10c76:7ad38071-84ee-4fc6-9a4a-32b58ad75e74::",
  "iam_apikey_name": "auto-generated-apikey-29929356-f5fb-44ac-9529-3f2447ffb6df",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/729dbd0074f3e622e009b8be41b10c76::serviceid:ServiceId-e1f7fe16-d35d-4af6-9dbb-a73534a299b4",
  "instance_id": "7ad38071-84ee-4fc6-9a4a-32b58ad75e74",
  "password": "b89a1c07-f657-4a0f-a5f3-e07ee9f57672",
  "url": "https://eu-de.ml.cloud.ibm.com",
  "username": "29929356-f5fb-44ac-9529-3f2447ffb6df"
}



In [24]:
# The code was removed by Watson Studio for sharing.

In [25]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### 4.2 Save the pipeline and model<a id="save"></a>

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [26]:
published_model_details = client.repository.store_model(model=model_rf, meta_props={'name':'Product line model'}, training_data=train_data, pipeline=pipeline_rf)

In [27]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

1bbf76a2-0896-4064-bdfc-78a902b50541


Get saved model metadata from Watson Machine Learning.

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available props.

In [28]:
client.repository.ModelMetaNames.show()

-----------------------  ----  --------
META_PROP NAME           TYPE  REQUIRED
NAME                     str   Y
DESCRIPTION              str   N
AUTHOR_NAME              str   N
FRAMEWORK_NAME           str   N
FRAMEWORK_VERSION        str   N
FRAMEWORK_LIBRARIES      list  N
RUNTIME_NAME             str   N
RUNTIME_VERSION          str   N
TRAINING_DATA_SCHEMA     dict  N
TRAINING_DATA_REFERENCE  dict  N
EVALUATION_METHOD        str   N
EVALUATION_METRICS       list  N
OUTPUT_DATA_SCHEMA       dict  N
LABEL_FIELD              str   N
TRANSFORMED_LABEL_FIELD  str   N
RUNTIME_UID              str   N
TRAINING_DEFINITION_URL  str   N
-----------------------  ----  --------


### 4.3 Load the model<a id="load"></a>

In this subsection you will learn how to load a saved model from a specified instance of Watson Machine Learning.

In [29]:
loaded_model = client.repository.load(model_uid)

You can print the model name to make sure that model has been loaded correctly.

In [30]:
print(type(loaded_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


As you can see the name is correct. 

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using the loaded model and visualize the prediction results with the plotly package.

- [5.1 Make a local prediction using previously loaded model and test data](#local)
- [5.2 Use Plotly to visualize data](#plotly)

### 5.1 Make a local prediction using previously loaded model and test data<a id="local"></a>

In this subsection you will score the *predict_data* data set.

In [31]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [32]:
predictions.show(5)

+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.31109657465052...|[0.26555482873252...|       1.0|Personal Accessories|
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.31109657465052...|[0.26555482873252...|       1.0|Personal Accessories|
|Camping E

By tabulating a count, you can see which product line is the most popular.

In [33]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

+------------------------+-----+
|predictedLabel          |count|
+------------------------+-----+
|Camping Equipment       |726  |
|Golf Equipment          |57   |
|Mountaineering Equipment|83   |
|Personal Accessories    |350  |
+------------------------+-----+



### 5.2 Use Plotly to visualize data <a id="plotly"></a>

In this subsection you will use the Plotly package to explore the prediction results. Plotly is an online analytics and data visualization tool.

First, you need to install the required packages. You can do it by running the following code. Run it one time only.

In [34]:
!pip install plotly
!pip install cufflinks==0.8.2

    100% |################################| 37.3MB 26kB/s  eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/30/4e/27c34b62430286c6d59177a0842ed90dc789ce5d1ed740887653b898779a/pytz-2018.5-py2.py3-none-any.whl
    100% |################################| 163kB 2.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/65/47/7e02164a2a3db50ed6d8a6ab1d6d60b69c4c3fdf57a284257925dfc12bda/requests-2.19.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
    100% |################################| 81kB 1.9MB/s eta 0:00:01
    100% |################################| 133kB 2.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bd/c9/6fdd990019071a4a32a5e7cb78a1d92c53851ef4f56f62a3486e6a7d8ffb/urllib3-1.23-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a9

  Using cached https://files.pythonhosted.org/packages/65/47/7e02164a2a3db50ed6d8a6ab1d6d60b69c4c3fdf57a284257925dfc12bda/requests-2.19.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/77/de/47e35a97b2b05c2fadbec67d44cfcdcd09b8086951b331d82de90d2912da/jsonschema-2.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/d6/abcb22de61d78e2fc3959c964628a5771e47e7cc60d53e9342e21ed6cc9a/traitlets-4.3.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/bc/9bd3b5c2b4774d5f33b2d544f1460be9df7df2fe42f352135381c347c69a/ipython_genutils-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1d/44/065d2d7bae7bebc06f1dd70d23c36da8c50c0f08b4236716743d706762a8/jupyter_core-4.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a97fab05a372e4a2c6a6b876165/idna-2.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.

Import Plotly and the other required packages.

In [39]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [40]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [41]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
product_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['GENDER'])]
product_layout = go.Layout(title='Predicted product line client interest distribution')

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [42]:
age_data = [go.Bar(y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"], x=cumulative_stats.index)]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(title = "Product Line", showline=False),
    yaxis=dict(title = "Mean AGE"))

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on the bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record using the `watson-machine-learning-client`.

**Note:** You can also use the REST API to deploy and score.
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

#### Create online deployment for published model.

In [43]:
deployment_details = client.deployments.create(model_uid, name='Product line model deployment')



#######################################################################################

Synchronous deployment creation for uid: '1bbf76a2-0896-4064-bdfc-78a902b50541' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='94353e9c-bd8f-441a-bf94-b911a0efe430'
------------------------------------------------------------------------------------------------




#### Create an online scoring endpoint. 

In [44]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://eu-de.ml.cloud.ibm.com/v3/wml_instances/7ad38071-84ee-4fc6-9a4a-32b58ad75e74/deployments/94353e9c-bd8f-441a-bf94-b911a0efe430/online


Now, you can send new scoring records (new data) for which you would like to get predictions. To do that, run the following sample code: 

In [45]:
payload_scoring = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}

client.deployments.score(scoring_url, payload_scoring)

{'fields': ['GENDER',
  'AGE',
  'MARITAL_STATUS',
  'PROFESSION',
  'PROFESSION_IX',
  'GENDER_IX',
  'MARITAL_STATUS_IX',
  'features',
  'rawPrediction',
  'probability',
  'prediction',
  'predictedLabel'],
 'values': [['M',
   23,
   'Single',
   'Student',
   6.0,
   0.0,
   1.0,
   [0.0, 23.0, 1.0, 6.0],
   [6.013441307221542,
    6.239511698689834,
    5.861319575359663,
    0.21877558715993115,
    1.666951831569029],
   [0.3006720653610771,
    0.31197558493449173,
    0.2930659787679831,
    0.010938779357996557,
    0.08334759157845145],
   1.0,
   'Personal Accessories'],
  ['M',
   55,
   'Single',
   'Executive',
   3.0,
   0.0,
   1.0,
   [0.0, 55.0, 1.0, 3.0],
   [4.62390427645954,
    5.374063582317748,
    3.252737939407842,
    6.643031519016991,
    0.10626268279787889],
   [0.231195213822977,
    0.2687031791158874,
    0.1626368969703921,
    0.33215157595084954,
    0.005313134139893944],
   3.0,
   'Golf Equipment']]}

As you can see we predict that a 23-year-old male student is interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0). You can also see that single, 55 years old man is interested in golf equipment.

<a id="summary"></a>
## 7. Summary and next steps     

You successfully completed this notebook! 
 
You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>